In [1]:
# ! pip install nmslib pandas torch requests streamlit
# ! conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
# ! pip install ftfy regex tqdm
# ! pip install git+https://github.com/openai/CLIP.git
# pip install googletrans==3.1.0a0

In [1]:
import os
import numpy as np
from nmslib_indexer import HnmsIndexer
import os
import glob
import numpy as np
from IPython.display import Image
import requests
imgs = sorted(glob.glob('frames/*'))

def show_img(i, imgs):
    return Image(filename=imgs[i])

files = sorted(glob.glob('frame_vectors/*'))
embs = np.vstack([np.load(x) for x in files])

indexer_path = 'indexer'
indexer = HnmsIndexer()

if not os.path.exists(indexer_path):
    indexer.add(embs)
    indexer.save(indexer_path)
    print('saved indexer')
else:
    indexer.load(indexer_path)
    print('loaded indexer')

# def search_video(query):
#     return 'videos/hb_nd_cv2.mp4'

import re

def get_id(img_path):
    return re.findall('.*\/(.*)_\d+.jpg', img_path)[0]

img_ids = dict()

for img_path in imgs:
    img_id = get_id(img_path)
    img_ids[img_id] = img_path
sorted(img_ids)

import os
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

def encode_text(predictor, text: str) -> np.ndarray:
    with torch.no_grad():
        text_latent = predictor.encode_text(clip.tokenize(text).to('cpu'))
        text_latent /= text_latent.norm(dim=-1, keepdim=True)
    return text_latent.cpu().detach().numpy()

text_ebm = [encode_text(model, 'pooping cat')]
scores, ids = indexer.find(text_ebm, 5)
print('scores:', scores)
print('ids:', ids)
[display(show_img(i, imgs)) for i in ids]
# show_img(183, imgs)

from generate_video_cut_queries import similar_frames_to_query

video_ids = [re.findall('.*/(.*)', imgs[i])[0] for i in ids]
gen_query = similar_frames_to_query(video_ids)
print(gen_query)


import os
from IPython.display import Video
from movie_dsl import MovDSL

# input_dir = 'downloaded'
input_dir = 'videos'
files = os.listdir(input_dir)
params = {file.replace(".mp4", ""): f'{input_dir}/{file}' for file in files}

dsl = MovDSL(params) \
    .query(gen_query) \
    .save(output_path)

Video(output_path)

loaded indexer


In [ ]:
# ! wget http://moments.csail.mit.edu/splits/Moments_in_Time_Raw_v2.zip -OMoments_in_Time_Raw_v2

In [5]:
import pandas as pd
df = pd.read_csv('data/results_10M_val.csv')
df.head()

,videoid,contentUrl,duration,page_dir,name
0,1023443014,https://ak.picdn.net/shutterstock/videos/10234...,PT00H00M30S,095201_095250,Disco light leaks disco ball light reflections...
1,1026421895,https://ak.picdn.net/shutterstock/videos/10264...,PT00H00M05S,199401_199450,Valle de la luna / san pedro de atacama / chil...
2,4350377,https://ak.picdn.net/shutterstock/videos/43503...,PT00H00M09S,137551_137600,Cloudy moscow kremlin time lapse
3,1054633538,https://ak.picdn.net/shutterstock/videos/10546...,PT00H00M19S,083701_083750,Sharp knife to cut delicious smoked fish
4,15702931,https://ak.picdn.net/shutterstock/videos/15702...,PT00H00M05S,138451_138500,The girl received flowers as a gift. a gift fo...


In [6]:
df['contentUrl'].values[1]

'https://ak.picdn.net/shutterstock/videos/1026421895/preview/stock-footage-valle-de-la-luna-san-pedro-de-atacama-chile-pull-focus-desert-and-salt.mp4'

In [7]:
from IPython.display import Video

def download_video_series(link, odir):
    os.makedirs(odir, exist_ok=True)
    file_name =  odir + '/' + link.split('/')[-1] 
    print( "downloading file:", file_name) 
    r = requests.get(link, stream = True) 
    with open(file_name, 'wb') as f: 
        for chunk in r.iter_content(chunk_size = 1024*1024): 
            if chunk: 
                f.write(chunk) 
    print('downloaded', file_name)
    return file_name
    
file_name = download_video_series(df['contentUrl'].values[0], 'downloaded')
Video(file_name)

downloading file: downloaded/stock-footage-disco-light-leaks-disco-ball-light-reflections-shaped-rectangular-and-line-with-motion-blur-effect.mp4
downloaded downloaded/stock-footage-disco-light-leaks-disco-ball-light-reflections-shaped-rectangular-and-line-with-motion-blur-effect.mp4


In [15]:
from googletrans import Translator

text = 'ехала Маша на машине сar'
translator = Translator()
text_translated = translator.translate(text, src = 'ru', dest='en').text
print(text, text_translated)

ехала Маша на машине сar Masha was driving a car


In [19]:
! pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.7 MB/s eta 0:00:00


In [32]:
%%time

from pytube import YouTube
# yt = YouTube("https://www.youtube.com/watch?v=n06H7OcPd-g")
# yt = yt.get('mp4', '720p')
# yt.download('downloaded')

def downloadYouTube(videourl, path):
    yt = YouTube(videourl)
    yt = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    yt.download(path)

# downloadYouTube('https://www.youtube.com/watch?v=zNyYDHCg06c', './videos/FindingNemo1')
e_Z6upbeYuc
downloadYouTube('https://youtu.be/yVV_t_Tewvs', 'downloaded/1.mp4')

CPU times: user 500 ms, sys: 86.1 ms, total: 586 ms
Wall time: 11.5 s


In [30]:
! du -h downloaded

4.0K	downloaded/.ipynb_checkpoints
51M	downloaded/1.mp4
54M	downloaded


In [31]:
Video('downloaded/1.mp4')

In [17]:
! ls downloaded

stock-footage-disco-light-leaks-disco-ball-light-reflections-shaped-rectangular-and-line-with-motion-blur-effect.mp4


In [38]:
! ls frames

ls: cannot access 'frames': No such file or directory


In [35]:
def add_audio(gen_query): 
    a = [x.strip() for x in gen_query.split('+')]
    return ' + '.join([f'{e} * {e.replace(".v", ".a")}' for e in a])

add_audio(gen_query)

'kotiki[61:68].v * kotiki[61:68].a + kotiki[245:251].v * kotiki[245:251].a + kotiki[13:14].v * kotiki[13:14].a + kotiki[187:197].v * kotiki[187:197].a'